In [24]:
import dgl
import torch 
from torch import nn
import numpy as np
import networkx as nx
from torch import FloatTensor as FT
from sklearn.preprocessing import OneHotEncoder

In [12]:
g = dgl.from_networkx(nx.karate_club_graph()) 
g.ndata['x'] = torch.ones(g.num_nodes(), 3) 
g.ndata['x'][1] = 0.5

In [15]:
from dgl import function as fn

In [17]:
fn.u_add_v

<function dgl.function.message._gen_message_builtin.<locals>.func(lhs_field, rhs_field, out)>

In [20]:
g.edata


{}

In [21]:
from dgl.utils import expand_as_pair, check_eq_shape

In [22]:
expand_as_pair

<function dgl.utils.internal.expand_as_pair(input_, g=None)>

In [29]:
ohe = OneHotEncoder()
features = ohe.fit_transform(np.arange(len(g.nodes())).reshape([-1,1])).todense()
features = np.array(features)
features

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [30]:
g.ndata['x'] = FT(
    features
)

In [31]:
g.ndata['x'][1] 

tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [128]:
class GNN_1(nn.Module):
    def __init__(
        self,
        in_feats,
        out_feats,
        aggregator_type,
        bias=True,
        norm=None,
        activation=None
    ):
        super(GNN_1, self).__init__()
        self._in_src_feats, self._in_dst_feats = expand_as_pair(in_feats)
        self._out_feats = out_feats
        self._aggre_type = aggregator_type
        self.norm = norm
        self.activation = activation
        
        self.fc_self = nn.Linear(self._in_dst_feats, out_feats, bias=bias)
        self.FC_w_1 = nn.Linear(self._in_src_feats*2, out_feats, bias=bias)
        self.FC_w_2 = nn.Linear(out_feats*2, out_feats, bias=bias)
        self.reset_parameters()
        self.aggregator_type = aggregator_type
        return
    
    def reset_parameters(self):
        """Reinitialize learnable parameters."""
        gain = nn.init.calculate_gain('relu')
        nn.init.xavier_uniform_(self.fc_self.weight, gain=gain)
        nn.init.xavier_uniform_(self.FC_w_1.weight, gain=gain)
        nn.init.xavier_uniform_(self.FC_w_2.weight, gain=gain)
        return
    
    
    def forward(self, graph, x):
        x_src, x_dst = expand_as_pair(x, graph)
        print(x_src.shape)
        graph.srcdata['h'] = x_src
        for layer in range(2):
            print('layer ',layer)
            graph.update_all(
                fn.copy_u('h', 'm'), 
                fn.mean('m', 'nbr')
            )
            h_self = graph.srcdata['h']
            h_neigh = graph.dstdata['nbr']
            # Concatenate
            rst = torch.cat([h_self, h_neigh],dim=1)
           
            if layer==0:
                rst = self.FC_w_1(rst)
            else:
                rst = self.FC_w_2(rst)
            print(rst.shape)
            graph.srcdata['h'] = rst
        # Do normalization
        graph.srcdata['h'] 
        return
    

In [129]:
gnn_1 = GNN_1(
    in_feats= 34,
    out_feats=8,
    aggregator_type='mean'
)

In [130]:
g.nodes()[:10]

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [131]:
gnn_1(
    g, g.ndata['x']
)

torch.Size([34, 34])
layer  0
torch.Size([34, 8])
layer  1
torch.Size([34, 8])
